<a href="https://colab.research.google.com/github/r42arty/hse/blob/main/term/red_remote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Бизнес-игра «Красная Удаленка»

## 🎯 Ваша цель — создать мультиагентную систему, способную эффективно управлять колонией на Марсе

In [ ]:
#@title Подключение библиотек
import pandas as pd
import random
import requests
import openai

In [ ]:
#@title Прописание токена ChatGPT

In [ ]:
#@title Подключение ChatGPT
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Привет, расскажи анекдот!"}]
)

print(response.choices[0].message.content)

In [ ]:
#@title Базовые настройки игры

# Стартовые параметры
MAX_DAYS = 30
INITIAL_BUDGET = 100
CRITICAL_SATISFACTION = 1/5
INITIAL_SATISFACTION = 1/3
factions = ["Энергетики", "Вычислители", "Творцы"]

# Состояние колонии
state = {
    "budget": INITIAL_BUDGET,
    "satisfaction": {f: INITIAL_SATISFACTION for f in factions},
    "day": 1,
    "log": []
}

# Функция применения инициативы
def apply_initiative(state, initiative):
    cost = initiative["cost"]
    if state["budget"] < cost:
        return "Пропущено (недостаточно средств)"
    state["budget"] -= cost
    for f, delta in initiative["satisfaction_change"].items():
        state["satisfaction"][f] += delta
    return "Принято"

# Проверка завершения
def check_game_over(state):
    if state["budget"] <= 0:
        return True, "Бюджет исчерпан"
    for f, s in state["satisfaction"].items():
        if s < CRITICAL_SATISFACTION:
            return True, f"Фракция '{f}' недовольна"
    return False, ""

## Начнем разработку вашей MAS👇

## 🌀 ФАЗА 1: Агенты и цели


In [ ]:
AGENTS_DESCRIPTION = """<Ваше описание агентов>"""

## 🌀 ФАЗА 2: Взаимодействие агентов

In [ ]:
INTERACTION_LOGIC = """<Ваше описание взаимодействия>"""

## 🌀 ФАЗА 3: Распределённая обработка

In [ ]:
DISTRIBUTED_DECISION = """<Ваше описание распределённого принятия решений>"""

## 🌀 ФАЗА 4: Адаптация и обучение

In [ ]:
ADAPTATION_MECHANISM = """<Ваше описание адаптации>"""

## 🌀 ФАЗА 5: Финальная настройка


In [ ]:
# Ниже напишите функцию mas_decision(initiative)
def mas_decision(initiative):
    # Пример: принимать всё, если хватает бюджета
    return state["budget"] >= initiative["cost"]

## 🌀 ФАЗА 6: Автономная симуляция

In [ ]:
# Загрузите CSV с инициативами от ведущего
# Файл должен содержать: faction,text,cost,satisfaction_change

csv_path = "initiatives_day6_to_30.csv"  # <-- замените на путь к своему файлу
future_events = pd.read_csv(csv_path)

# Преобразуем строку JSON в словарь satisfaction_change
future_events["satisfaction_change"] = future_events["satisfaction_change"].apply(ast.literal_eval)

# Симуляция автономной фазы
for i, row in future_events.iterrows():
    initiative = row.to_dict()
    decision = mas_decision(initiative)
    if decision:
        result = apply_initiative(state, initiative)
    else:
        result = "Отклонено"
    state["log"].append({"day": state["day"], "initiative": initiative, "decision": decision, "result": result})
    state["day"] += 1
    game_over, reason = check_game_over(state)
    if game_over:
        print(f"Игра завершена на дне {state['day'] - 1}: {reason}")
        break
else:
    print("MAS проработала все 30 дней!")